In [1]:
!pip install -q transformers sentence-transformers spacy protobuf==3.20.3
!pip install torch torch_geometric

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import pandas as pd
import numpy as np
import spacy
import ast
from typing import List, Dict
from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import pipeline
import torch_geometric


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 6.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
grain 0.2.15 requires protobuf>=5.28.3, but you have protobuf 3.20.3 which is incompatible.
onnx 1.20.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
ray 2.52.1 requires click!=8.3.*,>=7.0, but you have click 8.3.1 which is incompatible.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
bigframes 2.26.0 requires rich<14,>=12.4.

/usr/local/lib/python3.12/dist-packages/sqlalchemy/orm/query.py:195: SyntaxWarning: "is not" with 'tuple' literal. Did you mean "!="?
  if entities is not ():
E0000 00:00:1770194771.692929      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770194771.748387      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770194772.210364      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770194772.210409      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770194772.210412      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid lin

In [2]:
pairs_df = pd.read_csv("/kaggle/input/canndy/pairs_extracted.csv")
conceptnet_df = pd.read_csv("/kaggle/input/trycandy/conceptnet.en (1).csv",sep="\t",
    header=None)  # relation, head, tail, weight
conceptnet_df.columns = ["relation", "head", "tail", "weight"]
conceptnet_df



,relation,head,tail,weight
0,antonym,ab_extra,ab_intra,1.0
1,antonym,ab_intra,ab_extra,1.0
2,antonym,abactinal,actinal,1.0
3,antonym,abandon,acquire,1.0
4,antonym,abandon,arrogate,1.0
...,...,...,...,...
2487805,usedfor,zoom_lens,procure_better_shot,1.0
2487806,usedfor,zoom_lens,see_things_bigger,1.0
2487807,usedfor,zoom_lens,seeing_distant_object_more_closely,1.0
2487808,usedfor,zoom_lens,take_pictures,1.0


In [3]:
#pairs_df=pairs_df.head(8)

In [4]:
pairs_df

,qid,question,answer,qid.1,question.1,answer.1,concepts,numbers,units,relations
0,0,which is the suitable temperature for photosyn...,12\degree c - 25\degree c,0,which is the suitable temperature for photosyn...,12\degree c - 25\degree c,"['photosynthesis', 'temperature', 'suitable', ...",['12\\degree'],"['m', 'deg']",[]
1,0,which is the suitable temperature for photosyn...,17\degree c - 30\degree c,0,which is the suitable temperature for photosyn...,17\degree c - 30\degree c,"['photosynthesis', 'temperature', 'suitable', ...",['17\\degree'],"['m', 'deg']",[]
2,0,which is the suitable temperature for photosyn...,22\degree c - 35\degree c,0,which is the suitable temperature for photosyn...,22\degree c - 35\degree c,"['photosynthesis', 'temperature', 'suitable', ...",['22\\degree'],"['m', 'deg']",[]
3,0,which is the suitable temperature for photosyn...,27\degree c - 40\degree c,0,which is the suitable temperature for photosyn...,27\degree c - 40\degree c,"['photosynthesis', 'temperature', 'suitable']","['40\\degree', '27\\degree']","['m', 'deg']",[]
4,1,where is the solar energy stored as the potent...,oxygen,1,where is the solar energy stored as the potent...,oxygen,"['solar', 'energy', 'oxygen', 'potential', 'st...",[],[],[]
...,...,...,...,...,...,...,...,...,...,...
1399,349,what is the reason for the extinction of north...,struggle with environment,349,what is the reason for the extinction of north...,struggle with environment,"['reason', 'struggle', 'brid', 'south', 'envir...",[],['m'],[]
1400,350,which of the following species maintains chara...,mustard,350,which of the following species maintains chara...,mustard,"['mustard', 'follow', 'specie', 'reduce', 'abi...",[],['m'],[]
1401,350,which of the following species maintains chara...,papaya,350,which of the following species maintains chara...,papaya,"['papaya', 'follow', 'specie', 'reduce', 'abil...",[],['m'],[]
1402,350,which of the following species maintains chara...,shimul,350,which of the following species maintains chara...,shimul,"['shimul', 'follow', 'specie', 'reduce', 'abil...",[],['m'],[]


**Load data safely**

In [5]:
def ensure_list(x):
    if isinstance(x, list):
        return x
    if pd.isna(x):
        return []
    return ast.literal_eval(x)

pairs_df["concepts"] = pairs_df["concepts"].apply(ensure_list)

**Text normalization (for structured retrieval)**

In [6]:
#nlp = spacy.load("en_core_web_sm")

#def norm(text):
 #   return " ".join([t.lemma_.lower() for t in nlp(str(text))])
    
#conceptnet_df["head_norm"] = conceptnet_df["head"].apply(norm)
#conceptnet_df["tail_norm"] = conceptnet_df["tail"].apply(norm)
def norm(s: str) -> str:
    if not isinstance(s, str):
        s = str(s)
    return s.strip().lower()

**Bi-encoder (recall stage)**

In [7]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd

# 1️⃣ Make a safe copy of conceptnet_df if it might be a slice
conceptnet_df = conceptnet_df.copy()

# 2️⃣ Load bi-encoder on GPU
bi_encoder = SentenceTransformer(
    "all-MiniLM-L6-v2",
    device="cuda"
)

# 3️⃣ Ensure ConceptNet columns are clean strings safely
for col in ["head", "relation", "tail"]:
    conceptnet_df[col] = conceptnet_df[col].fillna("").astype(str)

# 4️⃣ Build edge text safely
conceptnet_df["edge_text"] = (
    conceptnet_df["head"].str.strip() + " " +
    conceptnet_df["relation"].str.strip() + " " +
    conceptnet_df["tail"].str.strip()
).str.strip()

# 5️⃣ Remove empty edges
conceptnet_df = conceptnet_df[conceptnet_df["edge_text"] != ""].reset_index(drop=True)

# 6️⃣ Encode ConceptNet edges (ONE-TIME cost)
edge_embeddings = bi_encoder.encode(
    conceptnet_df["edge_text"].tolist(),
    convert_to_numpy=True,
    show_progress_bar=True,
    batch_size=64
)

# 7️⃣ Save for reuse
np.save("conceptnet_edge_embeddings.npy", edge_embeddings)
conceptnet_df.to_csv("conceptnet_df_clean.csv", index=False)

print("✅ ConceptNet embeddings saved and ready for reuse.")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/38873 [00:00<?, ?it/s]

✅ ConceptNet embeddings saved and ready for reuse.
